In [1]:
import pydicom
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.exposure import cumulative_distribution
print("hi")

class ReadDICOMSeries(object):
    """
    Purpose: Reads an entire directory of DICOM images into 3D array.
    """

    def __init__(self, dirName):
        """
        Create list of files in dirName

        Inputs:
        dirName - Directory path for DICOM files
        """
        # Read DICOM scans list
        self.dirName = dirName
        self.dirList = os.listdir(dirName)

        # Initalize series dictonary
        ds = pydicom.dcmread(dirName + '/' + self.dirList[0])
        self.patientData                 = {}
        self.patientData["id"]           = ds[0x10,0x20].value
        self.patientData["rows"]         = int(ds[0x28,0x10].value)
        self.patientData["cols"]         = int(ds[0x28,0x11].value)
        self.patientData["voxel_size"]   = ds[0x18,0x50].value
        self.patientData["pixel_space"]  = ds[0x28,0x30].value
        self.patientData["acq_type"]     = ds[0x18,0x23].value
        self.patientData["smallest_pix"] = ds[0x28,0x106].value
        self.patientData["largest_pix"]  = ds[0x28,0x107].value
        self.patientData["numOfImages"]  = ds[0x20,0x1002].value

    def processImages(self):
        """
        Sorts images into order and resizes them if necessary.
        """
        sliceLoc  = np.zeros((self.patientData["numOfImages"],2))
        rawImages = np.zeros((self.patientData["numOfImages"], self.patientData["rows"], self.patientData["cols"]))
        Images    = np.zeros((self.patientData["numOfImages"], self.patientData["rows"], self.patientData["cols"]))

        for i in range(len(self.dirList)):
            ds = pydicom.dcmread(self.dirName + '/' + self.dirList[i])
            sliceLoc[i,0] = float(i)
            sliceLoc[i,1] = float(ds[0x20,0x1041].value)
            rawImages[i,:,:] = ds.pixel_array

        # Sort slice locations
        self.sliceloc = sliceLoc[sliceLoc[:,1].argsort()]
        for i in range(len(self.dirList)):
            Images[i,:,:] = rawImages[int(self.sliceloc[i,0]),:,:]

        # Do images need to be re-sized to 512x512
        if self.patientData["rows"] == 256 and self.patientData["cols"] == 256:
            newImages = np.zeros((self.patientData["numOfImages"],512,512))
            for t in range(self.patientData["numOfImages"]):
                newImages[t,:,:] = resize(Images[t,:,:], (512,512), mode='constant')

            Images = newImages
            self.sorted_imgs = newImages

        return Images

    def threshold(self):
        """
        Generates histogram of images for the 3D data
        """
        binNum = int(1 + 3.22*np.log(self.sorted_imgs.shape[0]*self.sorted_imgs.shape[1]*self.sorted_imgs.shape[2]))  # Sturge's Rule for bin number
        hist, bin_edges = np.histogram(self.sorted_imgs, bins=binNum)
        arg1 = np.min(np.argwhere(bin_edges>200))
        threshold = 2 + arg1 + np.argmax(hist[arg1:])
        return threshold


class ExtractImageValues(object):
    """
    Purpose: To view and extract data from images
    """
    def __init__(self, data, roi):
        self.data = data
        self.roi  = (roi-1)//2

    # The nothing function is necessary for the CV trackbar
    def nothing(self, x):
        pass

    def getData(self, image, p, numPoints):
        """
        """
        cs1 = CoordinateStore(image,p, self.roi)
        cv2.namedWindow('Image2')
        cv2.moveWindow('Image2', 1000,200)
        cv2.setMouseCallback('Image2', cs1.select_point)
        while(1):
            cv2.imshow('Image2',image)
            k = cv2.waitKey(20) & 0xFF
            if k == 27:
                mask = 255*np.zeros((9, image.shape[0], image.shape[1])).astype(np.uint8)
                #for t in range(numPoints):
                #    print(cs1.points[t][0], '    ', cs1.points[t][1], '   ', cs1.points[t][2])

                break
        #np.save(outDir + 'points_'+self.outName+'_Image_' + str(p) + '.npy', cs1.points)
        cv2.destroyWindow('Image2')
        return cs1.points


    def showImages(self):
        """
        Displays the images
        """
        self.imagePoints = np.zeros((0,3), dtype=np.int)
        imgs = np.zeros((self.data.shape[0],self.data.shape[1],self.data.shape[2],3)).astype(np.uint8)
        for t in range(imgs.shape[0]):
            imgs[t,:,:,0] = ((self.data[t,:,:]/(np.max(self.data[t,:,:]))*255)).astype(np.uint8)
            imgs[t,:,:,1] = ((self.data[t,:,:]/(np.max(self.data[t,:,:]))*255)).astype(np.uint8)
            imgs[t,:,:,2] = ((self.data[t,:,:]/(np.max(self.data[t,:,:]))*255)).astype(np.uint8)
        N = self.data.shape[0] - 1

        # Create window and trackbar
        cv2.namedWindow('image')
        cv2.moveWindow('image', 200, 200)
        cv2.createTrackbar('pos', 'image', 0, N, self.nothing)

        while(1):
            p = cv2.getTrackbarPos('pos','image')
            cv2.imshow('image',imgs[p,:,:,:])
            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                break
            elif k == 112:
                self.imagePoints = np.vstack((self.imagePoints, self.getData(imgs[p,:,:,:],p,4)))

        cv2.destroyAllWindows()

        return self.imagePoints



class CoordinateStore(object):
    """
    Stores the pixel coordinates received from the mouse click.
    """

    def __init__(self, image, p, roi):
        self.points = np.zeros((0,3), dtype=np.int)
        self.img = image
        self.p   = p
        self.roi = roi

    def select_point(self,event,x,y,flags,param):
            if event == cv2.EVENT_LBUTTONDOWN:
                cv2.circle(self.img,(x,y),1,(255,0,0),-1)
                cv2.rectangle(self.img, (x-self.roi,y-self.roi),(x+self.roi,y+self.roi), (255,0,0), 1)
                self.points = np.vstack((self.points,[self.p,x,y]))



class AssembleData(object):
    """
    Takes as input the two image files for T1+C and T1 as well as the 
    position data taken from ExtractImageValues
    -----------------------------------------------------------------
    Inputs:
    t1c_image    - The T1+C image
    t1_image     - The T1 image
    pointsList   - List of points from ExtractImageValues
    roi_size     - Size of the ROI you want projected around the points 
    """
    def __init__(self, t1c_image, t1_image, pointsList, roi_size):
        self.t1c     = t1c_image    # (Nx512x512) T1+C image set
        self.t1      = t1_image     # (Nx512x512) T1 image set
        self.points  = pointsList   # List of all points selected from image [image number, x, y] 
        self.roiSize = roi_size     # Size of ROI you want

    def extractData(self):
        """
        This will take the (Nx512x512) image sets for T1+C and T1 and create a
        third difference set (T1+C minus T1).  All data three sets will then be
        rescaled into the range -1 to 1. 
        A square ROI of size self.roiSize, centered around the point (x,y) for 
        image number p in self.points will then be extracted.
        """
        t1C_n = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))
        t1Sub_n = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))
        t1_n = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))

#        # Perform ZeroMeanNormalization
#        t1C_ZMN = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))
#        t1_ZMN = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))

        for t in range(self.t1c.shape[0]):
            datT1C = []
            datT1  = []
            for i in range(512):
                for j in range(512):
                    if self.t1c[t,j,i] > 120:
                        datT1C.append(self.t1c[t,j,i])
                    if self.t1[t,j,i] > 120:
                        datT1.append(self.t1[t,j,i])

            mat1 = np.array(datT1C)
            mat2 = np.array(datT1)
#            t1C_ZMN[t,:,:] = (self.t1c[t,:,:] - np.mean(mat1))/np.std(mat1)
#            t1_ZMN[t,:,:]  = (self.t1[t,:,:]  - np.mean(mat2))/np.std(mat2)
#
#
#        # Create subtracted image
#        # t1Sub = self.t1c - self.t1
#        t1Sub_ZMN = t1C_ZMN - t1_ZMN
#
#        # Rescale images -1 to 1
#        for t in range(self.t1c.shape[0]):
#            t1C_n[t,:,:]   = 2*(t1C_ZMN[t,:,:]   - np.min(t1C_ZMN[t,:,:]))/(np.max(t1C_ZMN[t,:,:])    - np.min(t1C_ZMN[t,:,:]))   - 1
#            t1Sub_n[t,:,:] = 2*(t1Sub_ZMN[t,:,:] - np.min(t1Sub_ZMN[t,:,:]))/(np.max(t1Sub_ZMN[t,:,:])- np.min(t1Sub_ZMN[t,:,:])) - 1
#            t1_n[t,:,:]    = 2*(t1_ZMN[t,:,:]    - np.min(t1_ZMN[t,:,:]))/(np.max(t1_ZMN[t,:,:])      - np.min(t1_ZMN[t,:,:]))    - 1

        # Get ROI
        t1c_ROI   = np.zeros((self.points.shape[0], self.roiSize, self.roiSize))
        t1Sub_ROI = np.zeros((self.points.shape[0], self.roiSize, self.roiSize))
        t1_ROI    = np.zeros((self.points.shape[0], self.roiSize, self.roiSize))
        x = int(self.roiSize//2 + 1)
        y = int(self.roiSize//2)
        for i in range(self.points.shape[0]):
            p   = self.points[i,0]
            row = self.points[i,2]
            col = self.points[i,1]
            t1c_ROI[i,:,:]    = t1C_n[p-x:p+y,row-x:row+y,col-x:col+y]
            t1Sub_ROI[i,:,:] = t1Sub_n[p-x:p+y,row-x:row+y,col-x:col+y]
            t1_ROI[i,:,:]     = t1_n[p-x:p+y,row-x:row+y,col-x:col+y]

        return t1c_ROI, t1Sub_ROI, t1_ROI



class AssembleData3d(object):
    """
    Takes as input the two image files for T1+C and T1 as well as the 
    position data taken from ExtractImageValues
    -----------------------------------------------------------------
    Inputs:
    t1c_image    - The T1+C image
    t1_image     - The T1 image
    pointsList   - List of points from ExtractImageValues
    roi_size     - Size of the ROI you want projected around the points 
    """
    def __init__(self, t1c_image, pointsList, roi_size):
        self.t1c     = t1c_image    # (Nx512x512) T1+C image set
        self.points  = pointsList   # List of all points selected from image [image number, x, y] 
        self.roiSize = roi_size     # Size of ROI you want

    def extractData(self):
        """
        This will take the (Nx512x512) image sets for T1+C and T1 and create a
        third difference set (T1+C minus T1).  All data three sets will then be
        rescaled into the range -1 to 1. 
        A square ROI of size self.roiSize, centered around the point (x,y) for 
        image number p in self.points will then be extracted.
        """
        t1C_n = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))
#        t1Sub_n = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))
#        t1_n = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))

        # Perform ZeroMeanNormalization
        t1C_ZMN = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))
#        t1_ZMN = np.zeros((self.t1c.shape[0], self.t1c.shape[1], self.t1c.shape[2]))

        for t in range(self.t1c.shape[0]):
            datT1C = []
#            datT1  = []
            for i in range(512):
                for j in range(512):
                    if self.t1c[t,j,i] > 120:
                        datT1C.append(self.t1c[t,j,i])
#                    if self.t1[t,j,i] > 120:
#                        datT1.append(self.t1[t,j,i])

            mat1 = np.array(datT1C)
#            mat2 = np.array(datT1)
            t1C_ZMN[t,:,:] = (self.t1c[t,:,:]) # - np.mean(mat1))/np.std(mat1)
#            t1_ZMN[t,:,:]  = (self.t1[t,:,:]  - np.mean(mat2))/np.std(mat2)


        # Create subtracted image
        # t1Sub = self.t1c - self.t1
#        t1Sub_ZMN = t1C_ZMN - t1_ZMN

#        # Rescale images -1 to 1
#        for t in range(self.t1c.shape[0]):
#            t1C_n[t,:,:]   = 2*(t1C_ZMN[t,:,:]   - np.min(t1C_ZMN[t,:,:]))/(np.max(t1C_ZMN[t,:,:])    - np.min(t1C_ZMN[t,:,:]))   - 1
#            t1Sub_n[t,:,:] = 2*(t1Sub_ZMN[t,:,:] - np.min(t1Sub_ZMN[t,:,:]))/(np.max(t1Sub_ZMN[t,:,:])- np.min(t1Sub_ZMN[t,:,:])) - 1
#            t1_n[t,:,:]    = 2*(t1_ZMN[t,:,:]    - np.min(t1_ZMN[t,:,:]))/(np.max(t1_ZMN[t,:,:])      - np.min(t1_ZMN[t,:,:]))    - 1

        # Get ROI
        t1c_ROI   = np.zeros((self.points.shape[0], self.roiSize, self.roiSize, self.roiSize))
        print('t1c_ROI = {0}'.format(t1c_ROI.shape))
#        t1Sub_ROI = np.zeros((self.points.shape[0], self.roiSize, self.roiSize))
#        t1_ROI    = np.zeros((self.points.shape[0], self.roiSize, self.roiSize))
        x = int(self.roiSize//2 + 1)
        y = int(self.roiSize//2)
        for i in range(self.points.shape[0]):
            p   = self.points[i,0]
            row = self.points[i,2]
            col = self.points[i,1]
            t1c_ROI[i,:,:,:]    = t1C_ZMN[p-x:p+y,row-x:row+y,col-x:col+y]
#            t1Sub_ROI[i,:,:] = t1Sub_n[p-x:p+y,row-x:row+y,col-x:col+y]
#            t1_ROI[i,:,:]     = t1_n[p-x:p+y,row-x:row+y,col-x:col+y]

        t1c_ROI_n = np.zeros((self.points.shape[0], self.roiSize, self.roiSize, self.roiSize))
#        for t in range(t1c_ROI.shape[0]):
#            t1c_ROI_n[t,:,:,:] = 2*((t1c_ROI[t,:,:,:] - np.min(t1c_ROI[t,:,:,:]))/(np.max(t1c_ROI[t,:,:,:]) - np.min(t1c_ROI[t,:,:,:]))) - 1

        return t1c_ROI 



def concatenateData(t1c, t1Sub, t1):
    """
    Takes the three ROI's and puts them together.
    """
    output = np.zeros((t1c.shape[0], t1c.shape[1], t1c.shape[2],3))
    for t in range(0,output.shape[0]):
        output[t,:,:,0] = t1c[t,:,:]
        output[t,:,:,1] = t1Sub[t,:,:]
        output[t,:,:,2] = t1[t,:,:]

    return output




    


hi


In [4]:
if __name__ == '__main__':
    print("started")
    input_name = 'Z:\SummerStudents\data\\underwood\\3dT1'
    output_name = 'regular_brain'
    roi         = 17

    points = np.zeros((0,3), dtype=np.int)
    rd1 = ReadDICOMSeries(input_name)
    img1 = rd1.processImages()
#    h1, b1,bNum = rd1.histogram()
#    print(h1)
#    arg1 = np.min(np.argwhere(b1>200))
#    num1 = 2 + arg1 + np.argmax(h1[arg1:])
#    print(int(b1[num1]))
#    x1 = np.linspace(b1[0],b1[-1],bNum)
#    p1 = plt.bar(x1, h1, width=30)
#    plt.show()
    eiv = ExtractImageValues(img1,roi)
    points = (eiv.showImages())
    dat = AssembleData3d(img1,points,roi)
    outArray = dat.extractData()
    np.save('test_' + output_name + '.npy', outArray)
    print(outArray.shape)
    print(points)
    np.save('test_' + output_name + '_pts.npy', points)

started
t1c_ROI = (5, 17, 17, 17)
(5, 17, 17, 17)
[[ 34 187 377]
 [ 68 154 341]
 [ 68 149 225]
 [ 97 175 362]
 [ 97 319 370]]
